# Übung 6: Transfer learning

Neuronal Netze benötigen oft eine große Menge an Trainingsdaten, damit es nicht zu overfitting kommt. Transfer learning erlaubt es, mit relativ geringen Datenmenge dennoch erfolgreiche große Netze zu trainieren. Dabei verwendet man ein bereits auf einen anderen Datensatz (z.b. ImageNet) vortrainiertes Netzwerk, und ersetzt nur das letzte Layer durch ein neues. In dieser Übung geht es darum, ein Netzwerk für die Erkennung von Geparden und Leoparden in der freien Wildbahn zu trainineren. 

## Daten laden

Lade die Daten hier herunter: http://tonic.imp.fu-berlin.de/cv_data/data.tar.gz

Die Daten wurde bereits in ein Trainings- und Validierungsset geteilt. Die Ordnerstruktur ist wie bei vielen Bildklassifierungsdatensetzen so aufgebaut. Es gibt zwei Unterordner für die Trainings- und Validierunsdaten. In diesen Ordnern liegen dann jeweils alle Bilder von einer Klasse in einem Unterordner mit dem Namen der Klasse.

Ein Beispiel: Die Trainingsbilder für die Klasse "cheetah" liegen in dem Unterordner train/cheetah

Diese Orderstruktur wird auch von dem in keras enhaltenen ImageDataGenerator unterstützt.

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, get_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

import os
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
batch_size = 32
image_input_size = (32, 32)
data_path = 'data/'

In [3]:
train_data_path = os.path.join(data_path, 'train')
val_data_path = os.path.join(data_path, 'val')

izw_classes = ('unknown', 'cheetah', 'leopard')

generator = ImageDataGenerator(horizontal_flip=True)
val_generator = ImageDataGenerator(horizontal_flip=False)

train_gen = generator.flow_from_directory(
    train_data_path, 
    target_size=image_input_size,
    classes=izw_classes,
    batch_size=batch_size)

val_gen = val_generator.flow_from_directory(
    val_data_path, 
    target_size=image_input_size,
    classes=izw_classes,
    batch_size=batch_size)

Found 17857 images belonging to 3 classes.
Found 1915 images belonging to 3 classes.


## Training ohne transfer learning

Trainiere zuerst ein kleines Classifer-Netzwerk ohne transfer learning. Falls du keine Grafikkarte hast, solltest du nicht die volle Auflösung (siehe Variable image_input_size) verwenden, da das Training sonst zu lange dauert. Eine Bildgröße von 32x32 Pixeln wäre zum Beispiel möglich.

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.utils.np_utils import to_categorical
    

#convert the traing set to categorical 
#number_of_classes = 3
#to_categorical_tensor(train_gen, number_of_classes)
#to_categorical_tensor(val_gen, number_of_classes)

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

model.add(Flatten())
#model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

# Compile model
epochs = 50
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 15, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 13, 32)        4640      
_________________________________________________________________
activation_2 (Activation)    (None, 28, 13, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 6, 16)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 4, 64)         9280      
__________

In [5]:
# Fit the model 
model.fit_generator(
        train_gen,
        steps_per_epoch=1800 // batch_size,
        epochs=50,
        validation_data=val_gen,
        validation_steps=250 // batch_size)


model.save_weights('wights_1.h5')

Epoch 1/50
56/56 [==============================] - 32s 573ms/step - loss: 6.1026 - acc: 0.6205 - val_loss: 4.7491 - val_acc: 0.7054
Epoch 2/50
56/56 [==============================] - 28s 508ms/step - loss: 5.8644 - acc: 0.6362 - val_loss: 4.7491 - val_acc: 0.7054
Epoch 3/50
56/56 [==============================] - 31s 552ms/step - loss: 5.9094 - acc: 0.6334 - val_loss: 4.7491 - val_acc: 0.7054
Epoch 4/50
56/56 [==============================] - 32s 566ms/step - loss: 5.5946 - acc: 0.6529 - val_loss: 4.7491 - val_acc: 0.7054
Epoch 5/50
56/56 [==============================] - 30s 539ms/step - loss: 5.6575 - acc: 0.6490 - val_loss: 4.7491 - val_acc: 0.7054
Epoch 6/50
56/56 [==============================] - 28s 508ms/step - loss: 5.3427 - acc: 0.6685 - val_loss: 4.7491 - val_acc: 0.7054
Epoch 7/50
56/56 [==============================] - 28s 495ms/step - loss: 5.4596 - acc: 0.6613 - val_loss: 4.7491 - val_acc: 0.7054
Epoch 8/50
56/56 [==============================] - 29s 509ms/step - 

ImportError: `save_weights` requires h5py.

Erstelle eine Confusion matrix basierend auf den Ausgaben des Netzes für die Validierungsdaten und berechne den ROC AUC für die Klasse cheetah. Du kannst hierfür optional die scikit-learn Bibliothek verwenden.

In [15]:
# TODO

## Pretrained network

Lade nun ein auf Imagenet vortrainiertes Netzwerk und klassifiziere damit die Validierungsdaten. Eine Anleitung für keras findest du hier: https://keras.io/applications

Du kannst selber entscheiden, welche Netzwerkarchitektur du verwendest.

In [17]:
# TODO

Da der ImageNet-Datensatz auch die Klassen cheetah und leopard enthält, können wir sogar ohne transfer learning das vortrainierte Netzwerk evaluieren. Interpretiere alle Klassen außer cheetah und leopard als unknown und berechne wie im vorherigen Schritt die Confusion matrix und den ROC AUC score für die Klasse cheetah.

In [18]:
# TODO

## Transfer learning

Das vortrainierte Netzwerk kann nun mit unseren Daten weitertrainiert werden. Ersetze dafür das letzte Layer in dem Netzwerk mit einem Dense Layer mit 3 Ausgaben für unsere Klassen cheetah, leopard und unknown. Du kannst selbst entscheiden, ob du nun das komplette Netzwerk mit trainierst oder nur das neu eingefügte, letzte Layer.

Auch hierfür kannst du dich wieder an der keras Anleitung orientieren: https://keras.io/applications

In [19]:
# TODO

Evaluiere das so trainierte Netzwerk wie in den letzten beiden Aufgaben.

In [20]:
# TODO

## Auswertung

Beschreibe kurz qualitativ die Resultate. Wie unterscheiden sich die trainierten Netzwerke, zum Beispiel im Bezug auf die Genauigkeit oder die Laufzeit? Welche Entscheidungen musstest du bei der Erfüllung der Aufgaben treffen und warum hast du dich für den von dir gewählten Weg entschieden?